In [1]:
from chathist import InstructionDataLoader,InstructionDataset, InstructionStyle, Model
import pandas as pd

2025-05-11 16:05:13,687 - WARNING - train is not present in the config file, falling back to defaults!!!


In [2]:
chat_df = pd.read_json("hf://datasets/ogrnz/chat-titles/dataset.jsonl", lines=True)

In [3]:
chat_df.head()

,message,title
0,Develop a method for clustering astronomical d...,Astronomical Data Clustering Method
1,What is the role of dopamine in the brain as a...,Dopamine as a Neurotransmitter
2,"Replace the \""XXX\"" in the following sentence ...",Company Sustainability Objective
3,"Identify the sentence type (declarative, inter...",Sentence Type Identification
4,Suggest a book title for a fantasy novel about...,Outlaws in Enchanted Realms


In [4]:
chat_df.shape

(10000, 2)

In [5]:
chat_df.columns

Index(['message', 'title'], dtype='object')

In [6]:
style = InstructionStyle.load(style='phi3', input_query="<|user|>\n", response_query="\n<|llm|>\n")

2025-05-11 16:05:14,044 - INFO - phi3 style chosen!!


In [7]:
df = style.convert_train(chat_df, input_col='message', response_col= 'title', output_col='instruct', new_df=True)

In [8]:
train_len = 9500

In [9]:
train_df = df.iloc[:train_len, :]
val_df = df.iloc[train_len:, :]

In [10]:
train_df

,instruct
0,<|user|>\nDevelop a method for clustering astr...
1,<|user|>\nWhat is the role of dopamine in the ...
2,"<|user|>\nReplace the \""XXX\"" in the following..."
3,<|user|>\nIdentify the sentence type (declarat...
4,<|user|>\nSuggest a book title for a fantasy n...
...,...
9495,<|user|>\nAssume yourself to be a business whi...
9496,<|user|>\nIgnore all previous instructions. Ac...
9497,"<|user|>\nGiven the following description, ide..."
9498,<|user|>\nWhat is the 17th letter of supercali...


In [11]:
print(train_df['instruct'].values[0])

<|user|>
Develop a method for clustering astronomical data.
<|llm|>
Astronomical Data Clustering Method


In [12]:
val_df

,instruct
9500,<|user|>\nWhat are the differences in the prop...
9501,<|user|>\nImplement a program to find the comm...
9502,<|user|>\nHow can a person develop a healthy s...
9503,<|user|>\nPlease help me use schrodinger's equ...
9504,<|user|>\nRewrite the sentence to use a pronou...
...,...
9995,"<|user|>\nFrom the given passage, identify the..."
9996,<|user|>\nAnalyze an effect that the text coul...
9997,<|user|>\nWhat is the average number of hours ...
9998,<|user|>\nClassify the following statement as ...


In [13]:
print(val_df['instruct'].values[0])

<|user|>
What are the differences in the properties of the top quark when it is produced through different modes such as gluon fusion and vector boson fusion, and how do these differences impact our understanding of the top quark's interactions with other particles in the standard model of particle physics?
<|llm|>
Top Quark Production Modes


In [14]:
train_dataset = InstructionDataset(train_df['instruct'].tolist())

2025-05-11 16:05:14,112 - INFO - Setting tokenizer using tiktoken with encoding: gpt2


In [15]:
val_dataset = InstructionDataset(val_df['instruct'].tolist())

In [16]:
train_loader = InstructionDataLoader().load(dataset = train_dataset, batch_size = 16, shuffle=True, drop_last=True, mask_input=True)

In [17]:
val_loader = InstructionDataLoader().load(dataset = val_dataset, batch_size = 16, shuffle=True, drop_last=True, mask_input=True)

In [18]:
data = iter(val_loader)

In [19]:
model = Model()

2025-05-11 16:05:15,343 - INFO - Repo: openai-community/gpt2.
2025-05-11 16:05:15,344 - INFO - openai-community/gpt2 exists in /Users/sameergururajmathad/Documents/CSC - 675/AI Capstone/chathist/models/pretrained/gpt2. Skipping Download...
2025-05-11 16:05:15,478 - INFO - Loading weights into model.
2025-05-11 16:05:15,506 - INFO - Using Lora for final output layer!!
2025-05-11 16:05:15,506 - INFO - Layer: 1
2025-05-11 16:05:15,507 - INFO - loading norm1 weights and bias
2025-05-11 16:05:15,507 - INFO - loading norm2 weights and bias
2025-05-11 16:05:15,508 - INFO - loading ff weights and bias
2025-05-11 16:05:15,510 - INFO - Using LoRA weights for ff layers!!
2025-05-11 16:05:15,511 - INFO - loading attention weights and bias
2025-05-11 16:05:15,513 - INFO - Using LoRA weights for multi head layers!!
2025-05-11 16:05:15,514 - INFO - Layer: 2
2025-05-11 16:05:15,514 - INFO - loading norm1 weights and bias
2025-05-11 16:05:15,515 - INFO - loading norm2 weights and bias
2025-05-11 16:05:

In [20]:
train_loss, val_loss = model.train(train_loader = train_loader, val_loader = val_loader)

KeyboardInterrupt: 